# Testing notebook

This notebook is meant for visualizing stuff and testing code. 

In [1]:
import numpy as np
import pandas as pd
import os
import warnings
from dataclasses import dataclass
from nltk.tokenize import word_tokenize
from collections import defaultdict, Counter
from contextlib import contextmanager
from typing import Generator
import demoji
warnings.filterwarnings("ignore", category=FutureWarning)

import spacy
from spacy.tokens import Doc, Span, Token
from spacy import displacy
from load_spacy import nlp

In [30]:

def load_data(data_dir:str) -> pd.DataFrame:
    """Loads a .jsonl file"""
    return pd.read_json(data_dir, lines=True)
    

test_docs = [
    "It was Jane's car that got stolen last night.",
    "It is my boss who will send me home today.",
    "It is me who tom saw today.",
    "It was your friend Suzy I met!",
    "Was it the boy in the hat who saw you?",
    "It wasn't the dog in the tree that ate the apple."
]



In [39]:
s = "it was Jane’s car that got stolen last night"
target1 = "(was (it) (car (Jane  (‘s) (stolen (that) (got) (night (last)))))"
target2 = "(was-be-VBD-Root (it-it-PRP-SUBJ) (car- (Jane  (‘s) (stolen (that) (got) (night (last)))))"
doc = nlp(s)

        
def parse_to_string(token):
    if token.n_lefts + token.n_rights > 0:
        children = ' '.join(parse_to_string(child) for child in token.children)
        return f"({token.text} {children})"
    else:
        return f"({token.text})"

def generate_dependency_string(sentence:Span):
    return parse_to_string(sentence.root)

for sent in doc.sents:
    print(generate_dependency_string(sent))


(was (it) (car (Jane (’s)) (stolen (that) (got) (night (last)))))


In [37]:
displacy.render(doc)

In [ ]:
doc = nlp("This is a 😁 string 😀. I am using emojis in this string!!! 🤪 :) :) >:)")

for token in doc:
    print(token.text)

In [48]:
FEATURE_REGISTER = {}

class Register:
    
    def __init__(self, func):
        self.func = func
        #FEATURE_REGISTER[func.__name__] = func
        
    def __name__(self):
        return self.func.__name__

        
    def __call__(self, doc):
        
        features = self.func(doc)
        print("hello")
        return self.add_one_to_func_output(features)
    
    
    def add_one_to_func_output(self, arg):
        return arg + 1
        

@Register
def func1(doc:int):
    return doc

@Register
def func2(doc:int):
    return doc


hello


2

In [54]:
FEATURE_REGISTER[func1.__name__] = func1


FEATURE_REGISTER["func1"]

AttributeError: 'Register' object has no attribute '__name__'